In [1]:
# SELECT DISTINCT ?person ?fam ?org ?orgO WHERE {
#     GRAPH <http://tbio.orient.cas.cz/chrj_pr> {
#         ?person tbio:isMemberOfFamily ?p .
#         ?p tbio:hasFamily ?fam .
#         FILTER(!sameterm(?fam, tbio:WufengLinFamily)) .
#     }
#     OPTIONAL {
#         ?person ?orgP ?orgEvt .
#         FILTER(sameterm(?orgP, tbio:attendsSchool) || sameterm(?orgP, tbio:hasEmployment) || sameterm(?orgP, tbio:hasMembership))
#         ?orgEvt ?orgAt ?org .
#         FILTER(sameterm(?orgAt, tbio:educationAt) || sameterm(?orgAt, tbio:employmentAt) || sameterm(?orgAt, tbio:hasMembershipAt))
#         ?org rdf:type ?orgO .
#     }
# }

In [2]:
import pandas as pd

In [23]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/all_org_classes/connections'

filename = 'personFamilyOrgPeriod.tsv'

read_filename = '{0}/{1}'.format(filepath, filename)

In [24]:
workDf = pd.read_csv(read_filename, delimiter='\t')
# workDf = workDf.dropna()
workDf.shape, workDf.head()

((13633, 4),
                            ?person                              ?fam  \
 0  <http://tbio.orient.cas.cz#林熊祥>  <http://tbio.orient.cas.cz#嚴復家系>   
 1  <http://tbio.orient.cas.cz#林熊祥>  <http://tbio.orient.cas.cz#嚴復家系>   
 2  <http://tbio.orient.cas.cz#林熊祥>  <http://tbio.orient.cas.cz#嚴復家系>   
 3  <http://tbio.orient.cas.cz#林熊祥>  <http://tbio.orient.cas.cz#嚴復家系>   
 4  <http://tbio.orient.cas.cz#林熊祥>  <http://tbio.orient.cas.cz#嚴復家系>   
 
                            ?org                                     ?orgO  
 0  <http://tbio.orient.cas.cz#>              <http://tbio.orient.cas.cz#>  
 1  <http://tbio.orient.cas.cz#>        <http://tbio.orient.cas.cz#Person>  
 2  <http://tbio.orient.cas.cz#>  <http://www.w3.org/2002/07/owl#Ontology>  
 3  <http://tbio.orient.cas.cz#>  <http://tbio.orient.cas.cz#Organization>  
 4  <http://tbio.orient.cas.cz#>      <http://tbio.orient.cas.cz#Position>  )

In [37]:
# SELECT DISTINCT ?s WHERE {
#     GRAPH ?graph {
#         ?s rdfs:subClassOf tbio:Division_Period .
#     }
# }

Division_Period = [
    'JapaneseColonialOrganization',
    'EarlyPostWarOrganization',
    'RepublicanOrganization',
    'BothPeriods',
    'PreRetrossecionRepublicanOrganization',
    'QingPeriodOrganization',
    'UnknownPeriodOrganization'
]

In [38]:
# SELECT DISTINCT ?s WHERE {
#     GRAPH ?graph {
#         ?s rdfs:subClassOf tbio:SocietalSector .
#     }
# }

SocietalSector = [
    'Police',
    'ArmedForces',
    'Hospital',
    'ReligiousOrganization',
    'MassMedia',
    'PublicAdministration',
    'PrivateBusiness',
    'VoluntaryAssociation',
    'Academia_Education',
    'StateBusiness',
    'CulturalOrganization',
    'Politics',
    'UnknownSector'    
]

In [42]:
workList = []
for idx in range(0, len(workDf)):
    row = workDf.loc[idx]

    person = str(row['?person']).replace("<http://tbio.orient.cas.cz#", "").replace(">", "")
    family = str(row['?fam']).replace("<http://tbio.orient.cas.cz#", "").replace(">", "")
    org = str(row['?org']).replace("<http://tbio.orient.cas.cz#", "").replace(">", "")
    orgO = str(row['?orgO']).replace("<http://tbio.orient.cas.cz#", "").replace(">", "")

    workList.append([person, family, org, orgO])

In [44]:
def getPeriod(INperson, INfamily, INorg):
    for row in workList:
        person = row[0]
        family = row[1]
        org = row[2]
        orgO = row[3]

        if person == INperson and family == INfamily and org == INorg:
            if orgO in Division_Period:
                return orgO
    return ''

def getSocietal(INperson, INfamily, INorg):
    for row in workList:
        person = row[0]
        family = row[1]
        org = row[2]
        orgO = row[3]

        if person == INperson and family == INfamily and org == INorg:
            if orgO in SocietalSector:
                return orgO
    return ''

resList = []
for row in workList:
    person = row[0]
    family = row[1]
    org = row[2]
    
    period = getPeriod(person, family, org)
    if period == '':
        continue
        
    societal = getSocietal(person, family, org)
    if societal == '':
        continue

    resList.append([person, family, period, societal])
    
print(len(resList), resList[0:5])

7178 [['林熊祥', '嚴復家系', 'JapaneseColonialOrganization', 'PrivateBusiness'], ['林熊祥', '嚴復家系', 'JapaneseColonialOrganization', 'PrivateBusiness'], ['林熊祥', '嚴復家系', 'JapaneseColonialOrganization', 'PrivateBusiness'], ['林熊祥', '嚴復家系', 'JapaneseColonialOrganization', 'PrivateBusiness'], ['林熊祥', '嚴復家系', 'JapaneseColonialOrganization', 'PrivateBusiness']]


In [46]:
orgDf = pd.DataFrame(resList, columns=['Person', 'FamilyName', 'Organization', 'SocietalSection'])
orgDf.drop_duplicates(keep='first', inplace=True)
orgDf.sort_values(by=['FamilyName', 'Person'], inplace=True)
orgDf.head()

,Person,FamilyName,Organization,SocietalSection
3817,何傳,何傳家系,BothPeriods,PublicAdministration
3819,何傳,何傳家系,JapaneseColonialOrganization,PublicAdministration
3821,何傳,何傳家系,JapaneseColonialOrganization,PrivateBusiness
3827,何傳,何傳家系,JapaneseColonialOrganization,VoluntaryAssociation
3855,何永,何傳家系,JapaneseColonialOrganization,PrivateBusiness


In [47]:
write_file_to = '{0}/{1}'.format(filepath, 'personFamilyOrgPeriod_20190512_v1.xlsx')
orgDf.to_excel(write_file_to, index=False)